In [1]:
import requests
from scraping.splatoon3_ink import list_all_links_recursive

In [3]:
all_links = list_all_links_recursive()
with open("links.txt", "w") as f:
    f.write("\n".join(all_links))

In [3]:
all_links

[]

In [5]:
import scrapy
import os
import json
from urllib.parse import urlparse

class JsonDownloadSpider(scrapy.Spider):
    name = "jsondownload"
    start_urls = all_links
    custom_settings = {
        'CONCURRENT_REQUESTS': 16,
        'DOWNLOAD_DELAY': 3
    }

    def parse(self, response):
        base_dir =  "data/splatoon3_ink"
        link = response.url
        filename = os.path.basename(urlparse(link).path)
        year, month, day = filename.split(".")[0].split("-")
        directory = f"{base_dir}/{year}/{month}/{day}"
        if "xrank" in filename:
            directory += "/xrank"
        path = f"{directory}/{filename}"
        if not os.path.exists(path):
            if not os.path.exists(directory):
                os.makedirs(directory)
            data = json.loads(response.text)
            with open(path, "w") as f:
                json.dump(data, f)
